In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# このexcelファイルではいろんなデータとその評価を乱数で作成している
#recommendの対象は0〜3の人
df = pd.read_excel('/Users/fujiwarayoshiki/Desktop/KALS/Python/recommendation/movie data.xlsx')

In [3]:
#平均値を入力する
df['ave'] = 0
for i in range(len(df.index)):
    arr = df.loc[i,:].values
    ave_i = np.nanmean(arr)
    df.loc[i,'ave'] = ave_i
    arr_n = df.loc[:,'ave'].values
df.head(10)

#ここは元のファイルがわかるように新しくaveのデータセットを作った方がよかった！

,Superman,ted,anpanman,harrypotter,doraemon,youtube,beautyand beast,lalaland,noah,predict1,predict2,predict3,predict4,ave
0,7,7,4,7,9,2,6,3,2,NaN,NaN,NaN,NaN,4.700000
1,4,9,6,1,8,1,6,2,1,NaN,NaN,NaN,NaN,3.800000
2,3,7,2,6,2,9,7,2,9,NaN,NaN,NaN,NaN,4.700000
3,2,8,5,7,1,3,7,1,9,NaN,NaN,NaN,NaN,4.300000
4,7,7,5,9,3,5,9,1,1,5.0,2.0,8.0,6.0,4.857143
5,7,2,9,5,9,2,6,2,3,5.0,2.0,4.0,7.0,4.500000
6,7,4,4,7,6,1,1,3,2,6.0,7.0,6.0,9.0,4.500000
7,5,2,6,5,7,3,5,1,5,8.0,6.0,1.0,6.0,4.285714
8,1,8,6,7,2,6,5,2,6,1.0,5.0,3.0,6.0,4.142857
9,7,9,3,7,7,8,7,3,5,4.0,4.0,7.0,2.0,5.214286


In [4]:
#辞書型に変換　人の番号をint表示
dataset = {}

for i in range(len(df.index)):

    d = dict(df.loc[i])
    
    dataset[int(i)] = d

In [5]:
def get_similarity(person1, person2):
    """
    params:
    """
    #欠損値の削除
    dataset = {}
    d = dict(df.loc[person1])
    dataset[person1] = d
    d = dict(df.loc[person2])
    dataset[person2] = d
    
    df_new = pd.DataFrame(dataset)
    df_new = df_new.T
    df_new = df_new.dropna(axis=1)

    
    dataset = {}
    d1 = dict(df_new.loc[person1])
    dataset[person1] = d1
    d2 = dict(df_new.loc[person2])
    dataset[person2] = d2
    
    
    
    # 両者とも見た映画の集合を取る
    set_person1 = set(dataset[person1].keys())
    set_person2 = set(dataset[person2].keys())
    set_both = set_person1.intersection(set_person2) #共通する要素からなる新しい集合

    if len(set_both)==0: #共通でみた映画がない場合は類似度を0とする
        return 0

    list_cov = []
    list_var1 = [] 
    list_var2 = []
  

    for item in set_both:
    # 同じ映画のレビュー点の差の2乗を計算
    # この数値が大きいほど「気が合わない」=「似ていない」と定義できる 
    
        cov = (dataset[person1][item]-df.loc[int(person1),'ave'])*(dataset[person2][item]-df.loc[int(person2),'ave'])
        var1 = (dataset[person1][item]-df.loc[int(person1),'ave'])**2
        var2 = (dataset[person2][item]-df.loc[int(person2),'ave'])**2
    
        list_cov.append(cov)
        list_var1.append(var1)
        list_var2.append(var2)

    return sum(list_cov)/((sum(list_var1)**(1/2))*(sum(list_var2)**(1/2)))

In [6]:
#類似度がminimum_similar_valueよりも大きい人をlistとして返す関数
def get_similar_person(person1 , minimum_similar_value):
    
    list_similar_person = []
    
    for i in range(len(df.index)):
        if i != person1 :
            
            if get_similarity(person1, i) >= minimum_similar_value :
                list_similar_person.append(i)

    return list_similar_person

In [7]:
#get_similar_personで出した、類似度が高い人を利用して、重み付き平均を用いて予測値を返す関数
def value_prediction(person1,minimum_similar_value,column):
    
    list_similarity_ave = []
    list_mass_ave = []
    
    for hoge in get_similar_person(person1 , minimum_similar_value):
        
        list_similarity_ave.append(get_similarity(person1, hoge))
        
        #比較する人の予想項目がnanの場合を除く
        if not(math.isnan(df.loc[hoge,column])) :
            list_mass_ave.append(get_similarity(person1,hoge)*(df.loc[hoge,column]-df.loc[hoge,'ave']))
        
    if len(list_similarity_ave)==0: #分母が0の場合を削除でnanを出力
        return np.nan
        
    else: 
        predicted_value = df.loc[person1,'ave'] + ((sum(list_mass_ave))/(sum(list_similarity_ave)))
        return predicted_value

In [10]:
#リコメンドする関数
def get_recommend ( person1,minimum_similar_value,recommend_num):
    
    
    list_person1 = [] #nanがついてるcolumnを取り出すためのリスト
    rankings = []

    #person1からnanのものを取ってくる
    for item in df.columns.values:
        if math.isnan(df.loc[person1,item]):
            list_person1.append(item)
            
    
    #そのnanについてvalueprediction
    rankings = []
    
    for column in list_person1:
        
        if not(math.isnan(value_prediction(person1,minimum_similar_value,column))):
            rankings.append ((value_prediction(person1,minimum_similar_value,column),column))
            
    rankings.sort()
    rankings.reverse()

    return [i[1] for i in rankings][:recommend_num]

#終始dfが使われているが、このdfはdfが定められていることを前提としている。
#dfも関数の引数にした方が良い

In [11]:
# 3さんのrecommendを類似度0.5以上で2個受け取る
get_recommend(3,0.5,2)

['predict2', 'predict3']

In [12]:
# minimum_similar_valueが高過ぎればnanを推薦なしとなる
get_recommend(1,0.9,2)

[]